In [ ]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import numpy as np
import pandas as pd

import pydicom as dicom
import os
import cv2
import PIL # optional
import shutil

In [ ]:
# Loading input data file in png format to dataframes
#train images in png format and stored in annotation format in dataframe
import os
kf = pd.DataFrame(columns=['data_type','StudyInstanceUID','image_file_ID','image_file_path'])
for dirname, _, filenames in os.walk('/kaggle/input/siim-covid19-detection-dicom-to-png-conversion'):
    for filename in filenames:
        
        std_id = filename.split('_')[0]
        #f_name_id = filename.split('_')[1].split('.')[0]
        dat_type = dirname.split('/')[-1]
        img_path = os.path.join(dirname, filename)
        
        kf.loc[kf.shape[0]] = [dat_type,std_id,filename,img_path]


png_data = pd.DataFrame(kf.drop([0,1,2,3,4],axis=0).values,columns=kf.columns)
png_data['image_file_ID'] = png_data.image_file_ID.str[13:-4]
png_data

# creating train and test datframes where all the image_study and studyInstanceUID are stored
png_test = png_data[png_data['data_type'] == 'test']
png_train = png_data[png_data['data_type'] == 'train']

In [ ]:
png_train

In [ ]:
png_test

In [ ]:
sample = pd.read_csv("/kaggle/input/siim-covid19-detection/sample_submission.csv")
train_image = pd.read_csv("/kaggle/input/siim-covid19-detection/train_image_level.csv")
train_study = pd.read_csv("/kaggle/input/siim-covid19-detection/train_study_level.csv")

# class label colum and case study id_number columns are created

train_study['classification_class'] = np.zeros((train_study.shape[0],1))
for i in range(0,train_study.shape[0]):
    train_study['classification_class'][i] = train_study.iloc[i].values[1:-1]

# StudyInstanceUID 
train_study['StudyInstanceUID'] = train_study.id.str[0:-6]
train_study

In [ ]:
itr = 0
cl = []
for i in train_study['classification_class'].values:
    if i[0]==1:
        cl.append('Negative')
    elif i[1]==1:
        cl.append('Typical')
    elif i[2]==1:
        cl.append('Indeterminate')
    elif i[3]==1:
        cl.append('Atypical')
    else:
        a=1
    itr = itr+1

train_study['class_type'] = cl
train_study

In [ ]:
# Selecting appropriate image for the StudyInstanceUID from coressponding multiple images_id's

SIUID = train_image.groupby('StudyInstanceUID') #SIUID :- Syudy instance UID indexed dataframe
im_fl_id = png_train.set_index('image_file_ID')

Image_file_path = []
Image_file_ID = []
for i in train_study['StudyInstanceUID'].values:
    
    pk = SIUID.get_group(i)
    if (pk.shape[0] >1) & (pk.dropna(subset=['boxes']).shape[0]>0):
        
        im_id = SIUID.get_group(i).dropna(subset=['boxes']).values[0][0].split('_')[0]
        path = im_fl_id.loc[im_id]['image_file_path']
        Image_file_path.append(path)
        Image_file_ID.append(im_id)
    
    else:
        im_id = SIUID.get_group(i).values[0][0].split('_')[0]
        path = im_fl_id.loc[im_id]['image_file_path']
        Image_file_path.append(path)
        Image_file_ID.append(im_id)

train_study['image_file_path'] = Image_file_path
train_study['image_file_ID'] = Image_file_ID
train_study

In [ ]:
# using this information for classification problem alone
train_study.info()

In [ ]:
# This data frame is for the object localisation and classification combined activities

train_Study_image = pd.merge(train_image,train_study,on='StudyInstanceUID',how="inner")
#
train_Study_image.columns = ['Image_id', 'boxes', 'label', 'StudyInstanceUID', 'Study_id',
       'Negative for Pneumonia', 'Typical Appearance',
       'Indeterminate Appearance', 'Atypical Appearance',
       'classification_class','class_type','image_file_path','image_file_ID']

train_Study_image

In [ ]:
train_Study_image.to_csv(r'Modified_train_study_Image_File_Study.csv')
pd.read_csv('Modified_train_study_Image_File_Study.csv')

In [ ]:
#Study Instance ID where all the images of the study ID are labelled as none
train_Study_image.set_index('StudyInstanceUID').loc['0d962bb4c9f2']

In [ ]:
path = [] 
for i in train_study['image_file_path'].values:
    
    pt = tf.keras.preprocessing.image.load_img(i)
    kt = tf.keras.preprocessing.image.img_to_array(pt)[40:230,40:230] # image cropping
    km = tf.keras.preprocessing.image.array_to_img(kt).resize([256,256])
    image_path_crp = '/kaggle/working' + '/'+ i.split('/')[-1]
    path.append(image_path_crp)
    km.save(image_path_crp)
    
train_study['Key_image_path'] = path

In [ ]:
train_study

In [ ]:
train_study.to_csv(r'Modified_train_study_Covid.csv')

In [ ]:
pd.read_csv('/kaggle/working/Modified_train_study_Covid.csv')